<a href="https://colab.research.google.com/github/ShriramGithub7/EVA_Custom_ResNet/blob/main/custom_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upgrade Albumentation package with latest version

In [1]:
!pip install --upgrade --force-reinstall --no-deps albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 KB 3.0 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.2.1
    Uninstalling albumentations-1.2.1:
      Successfully uninstalled albumentations-1.2.1


# git clone Master repo

In [2]:
!git clone https://github.com/ShriramGithub7/CNN-Master.git

Cloning into 'CNN-Master'...
remote: Enumerating objects: 377, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 377 (delta 82), reused 175 (delta 79), pack-reused 189
Receiving objects: 100% (377/377), 84.18 KiB | 3.24 MiB/s, done.
Resolving deltas: 100% (170/170), done.


# Import Data Transformation (Augumentation) code from git 

In [4]:
%cd /content/CNN-Master/utils
from dataTransform import *

trainset = AlbumentationImageDataset(train=True)
testset = AlbumentationImageDataset(train=False)

data = CIFAR10DataLoader(trainset, testset)

trainloader = data.train_loader
testloader = data.test_loader

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Check if CUDA is available

In [5]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available:", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)
    BATCH_SIZE=512
else:
    BATCH_SIZE=512

CUDA Available: True


In [6]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)


cuda


In [7]:
%cd /content/CNN-Master/
!touch /content/CNN-Master/model/__init__.py

/content/CNN-Master


# Import custom_resnet model from git and check the model parameters

In [8]:
!pip install torchsummary
from torchsummary import summary
from model.custom_resnet import *

net = CustomResNet().to(device)
summary(net, input_size=(3, 32, 32))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 32, 32]          73,728
         MaxPool2d-5          [-1, 128, 16, 16]               0
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
       BatchNorm2d-9          [-1, 128, 16, 16]             256
             ReLU-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]         147,584
      BatchNorm2d-12          [-1, 128, 16, 16]             256
    

In [9]:
%cd /content/CNN-Master/
from main import *

/content/CNN-Master


# Model Training after importing training code from master git repository

In [12]:
from main import ModelTrainer
trainer = ModelTrainer()
model, results = trainer.fit_model(net, trainloader, testloader, NUM_EPOCHS=24, l1=False, l2=False)


EPOCH: 1


Loss=-0.42 Batch_id=781 Accuracy=26.15: 100%|██████████| 782/782 [00:35<00:00, 22.22it/s]



Test set: Average loss: -0.3751, Accuracy: 3772/10000 (37.72%)

EPOCH: 2


Loss=-0.55 Batch_id=781 Accuracy=40.59: 100%|██████████| 782/782 [00:34<00:00, 22.63it/s]



Test set: Average loss: -0.4351, Accuracy: 4373/10000 (43.73%)

EPOCH: 3


...


Test set: Average loss: -0.7067, Accuracy: 7070/10000 (70.70%)

EPOCH: 23


Loss=-0.63 Batch_id=781 Accuracy=70.99: 100%|██████████| 782/782 [00:34<00:00, 22.75it/s]



Test set: Average loss: -0.7124, Accuracy: 7134/10000 (71.34%)

EPOCH: 24


Loss=-0.69 Batch_id=781 Accuracy=71.69: 100%|██████████| 782/782 [00:34<00:00, 22.75it/s]



Test set: Average loss: -0.7123, Accuracy: 7127/10000 (71.27%)

